# **Simple CNN**

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def calculate_mean_std_from_folder(dataset_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),          
    ])

    dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

    loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=2)

    mean = 0.0
    std = 0.0
    total_samples = 0

    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_samples += batch_samples

    mean /= total_samples
    std /= total_samples

    mean_list = mean.tolist()
    std_list = std.tolist()

    return mean_list, std_list



dataset_path = "new_bird"
mean_list, std_list = calculate_mean_std_from_folder(dataset_path)

print(f"Mean: {mean_list}")
print(f"Standard Deviation: {std_list}")


Mean: [0.4720446467399597, 0.4814448356628418, 0.4019118547439575]
Standard Deviation: [0.19218505918979645, 0.19124343991279602, 0.19615177810192108]


In [ ]:
import os
import shutil
import random

def split_dataset(input_folder, output_folder, train_ratio=0.75, valid_ratio=0.15, test_ratio=0.10):
    assert train_ratio + valid_ratio + test_ratio == 1.0, "Ratios must sum to 1.0"

    train_folder = os.path.join(output_folder, 'train')
    valid_folder = os.path.join(output_folder, 'valid')
    test_folder = os.path.join(output_folder, 'test')

    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(valid_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)
    for class_name in os.listdir(input_folder):
        class_path = os.path.join(input_folder, class_name)
        if not os.path.isdir(class_path):
            continue

        os.makedirs(os.path.join(train_folder, class_name), exist_ok=True)
        os.makedirs(os.path.join(valid_folder, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_folder, class_name), exist_ok=True)

        
        files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
        random.shuffle(files)

        # Split files into train, valid, and test sets
        train_split = int(len(files) * train_ratio)
        valid_split = train_split + int(len(files) * valid_ratio)

        train_files = files[:train_split]
        valid_files = files[train_split:valid_split]
        test_files = files[valid_split:]

        # Move files to their respective folders
        for file in train_files:
            shutil.copy(os.path.join(class_path, file), os.path.join(train_folder, class_name, file))
        for file in valid_files:
            shutil.copy(os.path.join(class_path, file), os.path.join(valid_folder, class_name, file))
        for file in test_files:
            shutil.copy(os.path.join(class_path, file), os.path.join(test_folder, class_name, file))

        print(f"Class '{class_name}' split into Train: {len(train_files)}, Valid: {len(valid_files)}, Test: {len(test_files)}")


input_folder = "new_bird"
output_folder = "bird"
split_dataset(input_folder, output_folder)


In [ ]:
import plotly.graph_objects as go

def plot_training_history(history):
    fig_acc = go.Figure()
    fig_acc.add_trace(go.Scatter(
        y=history['train_acc'], 
        mode='lines+markers', 
        name='Train Accuracy', 
        line=dict(color='red')
    ))
    fig_acc.add_trace(go.Scatter(
        y=history['valid_acc'], 
        mode='lines+markers', 
        name='Validation Accuracy', 
        line=dict(color='green')
    ))
    fig_acc.update_layout(
        title="Train vs Validation Accuracy",
        xaxis_title="Epoch",
        yaxis_title="Accuracy",
        legend_title="Legend",
    )
    fig_acc.show()

    # Loss Plot
    fig_loss = go.Figure()
    fig_loss.add_trace(go.Scatter(
        y=history['train_loss'], 
        mode='lines+markers', 
        name='Train Loss', 
        line=dict(color='blue')
    ))
    fig_loss.add_trace(go.Scatter(
        y=history['valid_loss'], 
        mode='lines+markers', 
        name='Validation Loss', 
        line=dict(color='yellow')
    ))
    fig_loss.update_layout(
        title="Train vs Validation Loss",
        xaxis_title="Epoch",
        yaxis_title="Loss",
        legend_title="Legend",
    )
    fig_loss.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 14 * 14)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
batch_size = 32

train_dataset = datasets.ImageFolder('bird/train', transform=transform)
val_dataset = datasets.ImageFolder('bird/valid', transform=transform)
test_dataset = datasets.ImageFolder('bird/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, loss function, and optimizer
num_classes = len(train_dataset.classes)
model = SimpleCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10
history_simple_cnn = {'train_acc': [], 'val_acc': [], 'train_loss': [], 'val_loss': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            t.set_postfix(loss=loss.item())
    train_acc = (correct / total)
    history_simple_cnn['train_loss'].append(train_loss / len(train_loader.dataset))
    history_simple_cnn['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    history_simple_cnn['val_loss'].append(val_loss / len(val_loader.dataset))
    history_simple_cnn['val_acc'].append(val_acc)

    print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {history_simple_cnn['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Val Loss: {history_simple_cnn['val_loss'][-1]:.4f}, Val Accuracy: {val_acc:.4f}")

# Test phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Test accuracy
test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")


Using device: cuda


Epoch 1/10: 100%|██████████| 879/879 [08:49<00:00,  1.66it/s, loss=1.54]


Epoch 1/10 - Train Loss: 2.2960, Train Accuracy: 0.2890, Val Loss: 1.7931, Val Accuracy: 0.4324


Epoch 2/10: 100%|██████████| 879/879 [08:44<00:00,  1.67it/s, loss=0.975]


Epoch 2/10 - Train Loss: 1.5294, Train Accuracy: 0.5228, Val Loss: 1.5096, Val Accuracy: 0.5269


...

Epoch 10/10: 100%|██████████| 879/879 [08:39<00:00,  1.69it/s, loss=0.215]  


Epoch 10/10 - Train Loss: 0.1556, Train Accuracy: 0.9504, Val Loss: 2.8371, Val Accuracy: 0.5820


Testing: 100%|██████████| 118/118 [01:11<00:00,  1.64it/s]



Test Accuracy: 0.5875



Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.57      0.69      0.62       150
      Brown-Headed-Barbet       0.37      0.45      0.40       150
             Cattle-Egret       0.77      0.65      0.70       150
        Common-Kingfisher       0.74      0.76      0.75       150
              Common-Myna       0.47      0.49      0.48       150
         Common-Rosefinch       0.67      0.52      0.59       150
        Common-Tailorbird       0.39      0.41      0.40       150
       Coppersmith-Barbet       0.56      0.55      0.56       150
           Forest-Wagtail       0.55      0.74      0.63       150
             Gray-Wagtail       0.58      0.55      0.56       150
                   Hoopoe       0.64      0.76      0.70       150
               House-Crow       0.68      0.45      0.54       150
     Indian-Grey-Hornbill       0.42      0.45      0.44       150
           Indian-Peacock       0.78 

# **Improved CNN**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird"
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")

# Data Transforms

transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Improved CNN Model
class ImprovedCNN(nn.Module):
    def __init__(self, num_classes):
        super(ImprovedCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 8 * 8, 1024), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(1024, 512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


# Initialize the model
num_classes = len(train_dataset.classes)
model = ImprovedCNN(num_classes).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training Loop with Fine-Tuning
num_epochs = 10
history_improved_cnn = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_improved_cnn['train_loss'].append(train_loss / len(train_loader.dataset))
    history_improved_cnn['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_improved_cnn['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_improved_cnn['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_improved_cnn['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_improved_cnn['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

    # Scheduler step
    scheduler.step()

# Testing Phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))


plot_training_history(history_improved_cnn)

Using device: cuda


Epoch 1/10 - Validation: 100%|██████████| 176/176 [01:43<00:00,  1.70it/s]


Epoch 1/10 - Train Loss: 2.7908, Train Accuracy: 0.1653, Valid Loss: 2.1087, Valid Accuracy: 0.3495


Epoch 2/10 - Validation: 100%|██████████| 176/176 [02:13<00:00,  1.32it/s]


Epoch 2/10 - Train Loss: 2.0984, Train Accuracy: 0.3566, Valid Loss: 1.6666, Valid Accuracy: 0.4898


...

Epoch 9/10 - Train Loss: 0.7064, Train Accuracy: 0.7836, Valid Loss: 0.6268, Valid Accuracy: 0.8027


Epoch 10/10 - Validation: 100%|██████████| 176/176 [01:52<00:00,  1.56it/s]


Epoch 10/10 - Train Loss: 0.6491, Train Accuracy: 0.7985, Valid Loss: 0.5757, Valid Accuracy: 0.8181


Testing: 100%|██████████| 118/118 [01:16<00:00,  1.55it/s]



Test Accuracy: 0.8197

Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.83      0.89      0.86       150
      Brown-Headed-Barbet       0.69      0.61      0.65       150
             Cattle-Egret       0.83      0.94      0.88       150
        Common-Kingfisher       0.90      0.93      0.91       150
              Common-Myna       0.95      0.87      0.91       150
         Common-Rosefinch       0.64      0.61      0.63       150
        Common-Tailorbird       0.76      0.58      0.66       150
       Coppersmith-Barbet       0.69      0.84      0.76       150
           Forest-Wagtail       0.80      0.90      0.85       150
             Gray-Wagtail       0.76      0.80      0.78       150
                   Hoopoe       0.94      0.85      0.90       150
               House-Crow       0.80      0.79      0.79       150
     Indian-Grey-Hornbill       0.62      0.66      0.64       150
           Ind

# **Simple Resnet-50**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import os
from sklearn.metrics import confusion_matrix, classification_report

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird" 
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")

transform_train = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


model = models.resnet50(pretrained=False)
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes) 
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training Loop
num_epochs = 10
history_simple_resnet50 = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_simple_resnet50['train_loss'].append(train_loss / len(train_loader.dataset))
    history_simple_resnet50['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_simple_resnet50['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_simple_resnet50['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_simple_resnet50['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_simple_resnet50['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

# Testing
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))


plot_training_history(history_simple_resnet50)


Using device: cuda


c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/10 - Validation: 100%|██████████| 176/176 [01:52<00:00,  1.56it/s]


Epoch 1/10 - Train Loss: 2.5614, Train Accuracy: 0.1951, Valid Loss: 2.3045, Valid Accuracy: 0.2761


Epoch 2/10 - Validation: 100%|██████████| 176/176 [01:54<00:00,  1.54it/s]


Epoch 2/10 - Train Loss: 1.9476, Train Accuracy: 0.3752, Valid Loss: 1.7712, Valid Accuracy: 0.4396


...

Epoch 9/10 - Train Loss: 0.4625, Train Accuracy: 0.8571, Valid Loss: 0.5411, Valid Accuracy: 0.8361


Epoch 10/10 - Validation: 100%|██████████| 176/176 [01:50<00:00,  1.59it/s]


Epoch 10/10 - Train Loss: 0.4048, Train Accuracy: 0.8743, Valid Loss: 0.4561, Valid Accuracy: 0.8596


Testing: 100%|██████████| 118/118 [01:14<00:00,  1.58it/s]



Test Accuracy: 0.8627

Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.96      0.87      0.91       150
      Brown-Headed-Barbet       0.79      0.70      0.74       150
             Cattle-Egret       0.84      0.93      0.88       150
        Common-Kingfisher       0.89      0.95      0.92       150
              Common-Myna       0.85      0.91      0.88       150
         Common-Rosefinch       0.85      0.75      0.79       150
        Common-Tailorbird       0.79      0.83      0.81       150
       Coppersmith-Barbet       0.98      0.77      0.87       150
           Forest-Wagtail       0.88      0.87      0.88       150
             Gray-Wagtail       0.89      0.84      0.86       150
                   Hoopoe       0.95      0.89      0.92       150
               House-Crow       0.82      0.85      0.84       150
     Indian-Grey-Hornbill       0.54      0.82      0.65       150
           Ind

# **Improved Resnet-50**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import os
from sklearn.metrics import confusion_matrix, classification_report

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird"
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")



transform_train = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = models.resnet50(pretrained=True)


for param in model.parameters():
    param.requires_grad = False


for param in model.layer4.parameters():  
    param.requires_grad = True
for param in model.fc.parameters():  
    param.requires_grad = True


num_classes = len(train_dataset.classes)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes)
)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training Loop
num_epochs = 10
history_improved_resnet50 = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_improved_resnet50['train_loss'].append(train_loss / len(train_loader.dataset))
    history_improved_resnet50['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_improved_resnet50['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_improved_resnet50['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_improved_resnet50['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_improved_resnet50['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

    # Scheduler step
    scheduler.step()

# Testing Phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

plot_training_history(history_improved_resnet50)



Using device: cuda


c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/10 - Validation: 100%|██████████| 176/176 [01:56<00:00,  1.51it/s]


Epoch 1/10 - Train Loss: 0.6794, Train Accuracy: 0.8174, Valid Loss: 0.1761, Valid Accuracy: 0.9486


Epoch 2/10 - Validation: 100%|██████████| 176/176 [01:52<00:00,  1.57it/s]


Epoch 2/10 - Train Loss: 0.2439, Train Accuracy: 0.9306, Valid Loss: 0.1374, Valid Accuracy: 0.9570


...

Epoch 9/10 - Train Loss: 0.0422, Train Accuracy: 0.9877, Valid Loss: 0.0594, Valid Accuracy: 0.9847


Epoch 10/10 - Validation: 100%|██████████| 176/176 [01:53<00:00,  1.55it/s]


Epoch 10/10 - Train Loss: 0.0417, Train Accuracy: 0.9870, Valid Loss: 0.0518, Valid Accuracy: 0.9845


Testing: 100%|██████████| 118/118 [01:17<00:00,  1.52it/s]



Test Accuracy: 0.9827

Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.98      0.96      0.97       150
      Brown-Headed-Barbet       0.94      0.99      0.96       150
             Cattle-Egret       0.99      0.99      0.99       150
        Common-Kingfisher       0.99      0.99      0.99       150
              Common-Myna       1.00      0.97      0.99       150
         Common-Rosefinch       0.98      0.93      0.95       150
        Common-Tailorbird       0.99      0.97      0.98       150
       Coppersmith-Barbet       0.97      0.97      0.97       150
           Forest-Wagtail       0.95      0.98      0.96       150
             Gray-Wagtail       0.99      0.99      0.99       150
                   Hoopoe       0.99      1.00      0.99       150
               House-Crow       0.97      0.97      0.97       150
     Indian-Grey-Hornbill       0.95      0.97      0.96       150
           Ind

# **Simple EfficientnetB0**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird"  
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")

# Data Transforms

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet expects 224x224 inputs
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Simple EfficientNet-B0
model = models.efficientnet_b0(pretrained=False)  # No pretrained weights
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)  # Replace classifier
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training Loop
num_epochs = 10
history_simple_efficientnetb0 = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_simple_efficientnetb0['train_loss'].append(train_loss / len(train_loader.dataset))
    history_simple_efficientnetb0['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_simple_efficientnetb0['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_simple_efficientnetb0['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_simple_efficientnetb0['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_simple_efficientnetb0['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

# Testing Phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

plot_training_history(history_simple_efficientnetb0)


Using device: cuda


c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.

Epoch 1/10 - Validation: 100%|██████████| 176/176 [01:53<00:00,  1.55it/s]


Epoch 1/10 - Train Loss: 2.4599, Train Accuracy: 0.1993, Valid Loss: 1.9570, Valid Accuracy: 0.3445


Epoch 2/10 - Validation: 100%|██████████| 176/176 [01:50<00:00,  1.59it/s]


Epoch 2/10 - Train Loss: 1.7391, Train Accuracy: 0.4226, Valid Loss: 1.6647, Valid Accuracy: 0.4793


...

Epoch 9/10 - Train Loss: 0.3664, Train Accuracy: 0.8822, Valid Loss: 0.4482, Valid Accuracy: 0.8677


Epoch 10/10 - Validation: 100%|██████████| 176/176 [02:01<00:00,  1.45it/s]


Epoch 10/10 - Train Loss: 0.3137, Train Accuracy: 0.8995, Valid Loss: 0.3367, Valid Accuracy: 0.8964


Testing: 100%|██████████| 118/118 [01:21<00:00,  1.45it/s]


Test Accuracy: 0.9000

Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.96      0.94      0.95       150
      Brown-Headed-Barbet       0.82      0.87      0.85       150
             Cattle-Egret       0.77      0.96      0.86       150
        Common-Kingfisher       0.95      0.95      0.95       150
              Common-Myna       0.97      0.92      0.95       150
         Common-Rosefinch       0.77      0.83      0.80       150
        Common-Tailorbird       0.91      0.77      0.83       150
       Coppersmith-Barbet       0.90      0.96      0.93       150
           Forest-Wagtail       0.98      0.83      0.90       150
             Gray-Wagtail       0.96      0.87      0.91       150
                   Hoopoe       0.93      0.93      0.93       150
               House-Crow       0.82      0.87      0.84       150
     Indian-Grey-Hornbill       0.74      0.78      0.76       150
           Ind

# **Improved EfficientnetB0**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird"
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")

# Data Transforms

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet expects 224x224 inputs
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Improved EfficientNet-B0 with Fine-Tuning
model = models.efficientnet_b0(pretrained=True)  # Use pretrained weights

# Freeze earlier layers for transfer learning
for param in model.features.parameters():
    param.requires_grad = False

# Fine-tune classifier
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes)
)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training Loop with Fine-Tuning
num_epochs = 10
history_improved_efficeintnetb0 = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_improved_efficeintnetb0['train_loss'].append(train_loss / len(train_loader.dataset))
    history_improved_efficeintnetb0['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_improved_efficeintnetb0['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_improved_efficeintnetb0['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_improved_efficeintnetb0['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_improved_efficeintnetb0['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

    # Scheduler step
    scheduler.step()

# Testing Phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

# Confusion Matrix
conf_matrix = confusion_matrix(all_labels, all_preds)
print(f"\nConfusion Matrix:\n{conf_matrix}")

plot_training_history(history_improved_efficeintnetb0)


Using device: cuda


Epoch 1/10: 100%|██████████| 440/440 [35:35<00:00,  4.85s/it]


Epoch 1/10 - Train Loss: 0.3868, Train Accuracy: 0.8921 - Val Loss: 0.1359, Val Accuracy: 0.9580


Epoch 2/10: 100%|██████████| 440/440 [35:41<00:00,  4.87s/it]


Epoch 2/10 - Train Loss: 0.1525, Train Accuracy: 0.9545 - Val Loss: 0.1340, Val Accuracy: 0.9607


...

Epoch 9/10 - Train Loss: 0.0196, Train Accuracy: 0.9942 - Val Loss: 0.0388, Val Accuracy: 0.9893


Epoch 10/10: 100%|██████████| 440/440 [32:56<00:00,  4.49s/it]


Epoch 10/10 - Train Loss: 0.0143, Train Accuracy: 0.9962 - Val Loss: 0.0320, Val Accuracy: 0.9916


Test Accuracy: 0.9909


Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       1.00      0.99      1.00       150
      Brown-Headed-Barbet       0.99      0.99      0.99       150
             Cattle-Egret       1.00      1.00      1.00       150
        Common-Kingfisher       1.00      1.00      1.00       150
              Common-Myna       1.00      0.99      0.99       150
         Common-Rosefinch       0.97      0.98      0.98       150
        Common-Tailorbird       0.99      0.99      0.99       150
       Coppersmith-Barbet       0.99      1.00      0.99       150
           Forest-Wagtail       0.98      0.98      0.98       150
             Gray-Wagtail       0.99      0.99      0.99       150
                   Hoopoe       1.00      1.00      1.00       150
               House-Crow       0.99      0.97      0.98       150
     Indian-Grey-Hornbill       0.95      0.97      0.96       150
           Indian-Peacock       0.98  

# **Simple MobilenetV2**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird"
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")

# Data Transforms

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # MobileNet expects 224x224 inputs
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Simple MobileNetV2
model = models.mobilenet_v2(pretrained=False)  # No pretrained weights
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)  # Replace classifier
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training Loop
num_epochs = 10
history_simple_mobilenetv2 = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_simple_mobilenetv2['train_loss'].append(train_loss / len(train_loader.dataset))
    history_simple_mobilenetv2['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_simple_mobilenetv2['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_simple_mobilenetv2['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_simple_mobilenetv2['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_simple_mobilenetv2['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

# Testing Phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

plot_training_history(history_simple_mobilenetv2)


Using device: cuda


c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/10 - Validation: 100%|██████████| 176/176 [01:50<00:00,  1.59it/s]


Epoch 1/10 - Train Loss: 2.3578, Train Accuracy: 0.2404, Valid Loss: 2.1074, Valid Accuracy: 0.3051


Epoch 2/10 - Validation: 100%|██████████| 176/176 [01:49<00:00,  1.60it/s]


Epoch 2/10 - Train Loss: 1.7468, Train Accuracy: 0.4306, Valid Loss: 2.0416, Valid Accuracy: 0.4174


...

Epoch 9/10 - Train Loss: 0.4859, Train Accuracy: 0.8463, Valid Loss: 0.5325, Valid Accuracy: 0.8404


Epoch 10/10 - Validation: 100%|██████████| 176/176 [01:49<00:00,  1.61it/s]


Epoch 10/10 - Train Loss: 0.4357, Train Accuracy: 0.8627, Valid Loss: 0.5123, Valid Accuracy: 0.8446


Testing: 100%|██████████| 118/118 [01:13<00:00,  1.61it/s]



Test Accuracy: 0.8496

Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.92      0.88      0.90       150
      Brown-Headed-Barbet       0.78      0.74      0.76       150
             Cattle-Egret       0.86      0.91      0.89       150
        Common-Kingfisher       0.96      0.92      0.94       150
              Common-Myna       0.81      0.91      0.86       150
         Common-Rosefinch       0.79      0.77      0.78       150
        Common-Tailorbird       0.86      0.65      0.74       150
       Coppersmith-Barbet       0.93      0.74      0.82       150
           Forest-Wagtail       0.73      0.92      0.81       150
             Gray-Wagtail       0.89      0.91      0.90       150
                   Hoopoe       0.93      0.93      0.93       150
               House-Crow       0.67      0.91      0.77       150
     Indian-Grey-Hornbill       0.68      0.63      0.66       150
           Ind

# **Improved MobilenetV2**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird"
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")

# Data Transforms

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Improved MobileNetV2 with Fine-Tuning
model = models.mobilenet_v2(pretrained=True)  # Use pretrained weights

# Freeze earlier layers for transfer learning
for param in model.features.parameters():
    param.requires_grad = False

# Fine-tune classifier
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes)
)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training Loop with Fine-Tuning
num_epochs = 10
history_improved_mobilenetV2 = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_improved_mobilenetV2['train_loss'].append(train_loss / len(train_loader.dataset))
    history_improved_mobilenetV2['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_improved_mobilenetV2['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_improved_mobilenetV2['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_improved_mobilenetV2['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_improved_mobilenetV2['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

    # Scheduler step
    scheduler.step()

# Testing Phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

plot_training_history(history_improved_mobilenetV2)


Using device: cuda


c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.

Epoch 1/10 - Validation: 100%|██████████| 176/176 [01:47<00:00,  1.63it/s]


Epoch 1/10 - Train Loss: 2.0048, Train Accuracy: 0.4675, Valid Loss: 0.9448, Valid Accuracy: 0.7801


Epoch 2/10 - Validation: 100%|██████████| 176/176 [01:48<00:00,  1.61it/s]


Epoch 2/10 - Train Loss: 1.1652, Train Accuracy: 0.6670, Valid Loss: 0.6874, Valid Accuracy: 0.8132


...

Epoch 9/10 - Train Loss: 0.7987, Train Accuracy: 0.7561, Valid Loss: 0.4887, Valid Accuracy: 0.8528


Epoch 10/10 - Validation: 100%|██████████| 176/176 [01:48<00:00,  1.62it/s]


Epoch 10/10 - Train Loss: 0.7826, Train Accuracy: 0.7619, Valid Loss: 0.4839, Valid Accuracy: 0.8452


Testing: 100%|██████████| 118/118 [01:13<00:00,  1.60it/s]


Test Accuracy: 0.8640

Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.81      0.86      0.83       150
      Brown-Headed-Barbet       0.77      0.67      0.71       150
             Cattle-Egret       0.93      0.93      0.93       150
        Common-Kingfisher       0.92      0.95      0.93       150
              Common-Myna       0.89      0.89      0.89       150
         Common-Rosefinch       0.75      0.77      0.76       150
        Common-Tailorbird       0.85      0.71      0.77       150
       Coppersmith-Barbet       0.84      0.83      0.84       150
           Forest-Wagtail       0.82      0.88      0.85       150
             Gray-Wagtail       0.78      0.86      0.82       150
                   Hoopoe       0.87      0.93      0.90       150
               House-Crow       0.90      0.83      0.87       150
     Indian-Grey-Hornbill       0.69      0.82      0.75       150
           Ind

# **Simple Densenet-121**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird"
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")

# Data Transforms

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # DenseNet expects 224x224 inputs
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Simple DenseNet121
model = models.densenet121(pretrained=False)  # No pretrained weights
num_classes = len(train_dataset.classes)
model.classifier = nn.Linear(model.classifier.in_features, num_classes)  # Replace classifier
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training Loop
num_epochs = 10
history_simple_densenet121 = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_simple_densenet121['train_loss'].append(train_loss / len(train_loader.dataset))
    history_simple_densenet121['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_simple_densenet121['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_simple_densenet121['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_simple_densenet121['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_simple_densenet121['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

# Testing Phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

plot_training_history(history_simple_densenet121)


Using device: cuda


Epoch 1/10 - Validation: 100%|██████████| 176/176 [04:30<00:00,  1.54s/it]


Epoch 1/10 - Train Loss: 2.1984, Train Accuracy: 0.3059, Valid Loss: 1.8700, Valid Accuracy: 0.4052


Epoch 2/10 - Validation: 100%|██████████| 176/176 [04:28<00:00,  1.52s/it]


Epoch 2/10 - Train Loss: 1.4132, Train Accuracy: 0.5502, Valid Loss: 1.1052, Valid Accuracy: 0.6494


...

Epoch 9/10 - Train Loss: 0.2879, Train Accuracy: 0.9098, Valid Loss: 0.3948, Valid Accuracy: 0.8802


Epoch 10/10 - Validation: 100%|██████████| 176/176 [04:29<00:00,  1.53s/it]


Epoch 10/10 - Train Loss: 0.2607, Train Accuracy: 0.9157, Valid Loss: 0.3172, Valid Accuracy: 0.9063


Testing: 100%|██████████| 118/118 [03:01<00:00,  1.54s/it]



Test Accuracy: 0.9117

Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.97      0.94      0.95       150
      Brown-Headed-Barbet       0.84      0.87      0.85       150
             Cattle-Egret       0.96      0.91      0.94       150
        Common-Kingfisher       0.99      0.95      0.97       150
              Common-Myna       0.96      0.95      0.95       150
         Common-Rosefinch       0.89      0.76      0.82       150
        Common-Tailorbird       0.78      0.93      0.84       150
       Coppersmith-Barbet       0.99      0.88      0.93       150
           Forest-Wagtail       0.94      0.93      0.93       150
             Gray-Wagtail       0.93      0.94      0.93       150
                   Hoopoe       1.00      0.96      0.98       150
               House-Crow       0.79      0.88      0.83       150
     Indian-Grey-Hornbill       0.67      0.85      0.75       150
           Ind

# **Improved Densenet-121**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird"
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")

# Data Transforms
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Improved DenseNet121 with Fine-Tuning
model = models.densenet121(pretrained=True)  # Use pretrained weights

# Freeze earlier layers for transfer learning
for param in model.features.parameters():
    param.requires_grad = False

# Fine-tune classifier
num_classes = len(train_dataset.classes)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes)
)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training Loop with Fine-Tuning
num_epochs = 10
history_improved_densenet121 = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_improved_densenet121['train_loss'].append(train_loss / len(train_loader.dataset))
    history_improved_densenet121['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_improved_densenet121['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_improved_densenet121['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_improved_densenet121['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_improved_densenet121['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

    # Scheduler step
    scheduler.step()

# Testing Phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

plot_training_history(history_improved_densenet121)


Using device: cuda


c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.

Epoch 1/10: 100%|██████████| 879/879 [1:38:05<00:00,  6.70s/it]


Epoch 1/10 - Train Loss: 0.6486, Train Accuracy: 0.8132 - Val Loss: 0.5796, Val Accuracy: 0.8199


Epoch 2/10: 100%|██████████| 879/879 [1:45:17<00:00,  7.19s/it]


Epoch 2/10 - Train Loss: 0.3603, Train Accuracy: 0.8940 - Val Loss: 0.3580, Val Accuracy: 0.8926


...

Epoch 9/10 - Train Loss: 0.0465, Train Accuracy: 0.9874 - Val Loss: 0.0581, Val Accuracy: 0.9835


Epoch 10/10: 100%|██████████| 879/879 [2:14:48<00:00,  9.20s/it]  


Epoch 10/10 - Train Loss: 0.0336, Train Accuracy: 0.9904 - Val Loss: 0.0511, Val Accuracy: 0.9861


Test Accuracy: 0.9829


Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.99      0.97      0.98       150
      Brown-Headed-Barbet       0.95      0.95      0.95       150
             Cattle-Egret       0.99      0.99      0.99       150
        Common-Kingfisher       1.00      1.00      1.00       150
              Common-Myna       0.99      0.98      0.99       150
         Common-Rosefinch       0.97      0.95      0.96       150
        Common-Tailorbird       0.99      0.99      0.99       150
       Coppersmith-Barbet       1.00      0.99      0.99       150
           Forest-Wagtail       0.94      0.98      0.96       150
             Gray-Wagtail       0.96      1.00      0.98       150
                   Hoopoe       0.99      0.99      0.99       150
               House-Crow       0.97      0.97      0.97       150
     Indian-Grey-Hornbill       0.95      0.93      0.94       150
           Indian-Peacock       0.98  

# **Simple Squeeznet**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird"
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")

# Data Transforms

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # SqueezeNet expects 224x224 inputs
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Simple SqueezeNet
model = models.squeezenet1_0(pretrained=False)  # No pretrained weights
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=1)  # Replace classifier
model.num_classes = num_classes
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training Loop
num_epochs = 10
history_simple_squeezenet = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_simple_squeezenet['train_loss'].append(train_loss / len(train_loader.dataset))
    history_simple_squeezenet['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_simple_squeezenet['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_simple_squeezenet['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_simple_squeezenet['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_simple_squeezenet['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

# Testing Phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

plot_training_history(history_simple_squeezenet)


Using device: cuda


c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.

Epoch 1/10 - Validation: 100%|██████████| 176/176 [03:08<00:00,  1.07s/it]


Epoch 1/10 - Train Loss: 3.2201, Train Accuracy: 0.0397, Valid Loss: 3.2189, Valid Accuracy: 0.0400


Epoch 2/10 - Validation: 100%|██████████| 176/176 [02:54<00:00,  1.01it/s]


Epoch 2/10 - Train Loss: 3.2189, Train Accuracy: 0.0390, Valid Loss: 3.2189, Valid Accuracy: 0.0400


...

Epoch 9/10 - Train Loss: 3.2189, Train Accuracy: 0.0403, Valid Loss: 3.2189, Valid Accuracy: 0.0400


Epoch 10/10 - Validation: 100%|██████████| 176/176 [02:57<00:00,  1.01s/it]


Epoch 10/10 - Train Loss: 3.2189, Train Accuracy: 0.0405, Valid Loss: 3.2189, Valid Accuracy: 0.0400


Testing: 100%|██████████| 118/118 [01:57<00:00,  1.01it/s]
c:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.




Test Accuracy: 0.0400

Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.04      1.00      0.08       150
      Brown-Headed-Barbet       0.00      0.00      0.00       150
             Cattle-Egret       0.00      0.00      0.00       150
        Common-Kingfisher       0.00      0.00      0.00       150
              Common-Myna       0.00      0.00      0.00       150
         Common-Rosefinch       0.00      0.00      0.00       150
        Common-Tailorbird       0.00      0.00      0.00       150
       Coppersmith-Barbet       0.00      0.00      0.00       150
           Forest-Wagtail       0.00      0.00      0.00       150
             Gray-Wagtail       0.00      0.00      0.00       150
                   Hoopoe       0.00      0.00      0.00       150
               House-Crow       0.00      0.00      0.00       150
     Indian-Grey-Hornbill       0.00      0.00      0.00       150
           Ind

# **Improved Squeezenet**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset structure
dataset_dir = "bird"
train_dir = os.path.join(dataset_dir, "train")
valid_dir = os.path.join(dataset_dir, "valid")
test_dir = os.path.join(dataset_dir, "test")

# Data Transforms

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_list, std_list)
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_test)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Improved SqueezeNet with Fine-Tuning
model = models.squeezenet1_0(pretrained=True)  # Use pretrained weights

# Fine-tune classifier
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Sequential(
    nn.Conv2d(512, num_classes, kernel_size=1),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d(1)
)
model.num_classes = num_classes
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training Loop with Fine-Tuning
num_epochs = 10
history_improved_squeezenet = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    history_improved_squeezenet['train_loss'].append(train_loss / len(train_loader.dataset))
    history_improved_squeezenet['train_acc'].append(train_acc)

    # Validation phase
    model.eval()
    valid_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    valid_acc = correct / total
    history_improved_squeezenet['valid_loss'].append(valid_loss / len(valid_loader.dataset))
    history_improved_squeezenet['valid_acc'].append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {history_improved_squeezenet['train_loss'][-1]:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Valid Loss: {history_improved_squeezenet['valid_loss'][-1]:.4f}, "
          f"Valid Accuracy: {valid_acc:.4f}")

    # Scheduler step
    scheduler.step()

# Testing Phase
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = (np.array(all_preds) == np.array(all_labels)).mean()
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

plot_training_history(history_improved_squeezenet)


Using device: cuda


c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

c:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.

Epoch 1/10 - Validation: 100%|██████████| 176/176 [03:56<00:00,  1.34s/it]


Epoch 1/10 - Train Loss: 1.4545, Train Accuracy: 0.5747, Valid Loss: 0.5621, Valid Accuracy: 0.8304


Epoch 2/10 - Validation: 100%|██████████| 176/176 [03:59<00:00,  1.36s/it]


Epoch 2/10 - Train Loss: 0.6453, Train Accuracy: 0.8088, Valid Loss: 0.4393, Valid Accuracy: 0.8629


...

Epoch 9/10 - Train Loss: 0.2269, Train Accuracy: 0.9302, Valid Loss: 0.1945, Valid Accuracy: 0.9424


Epoch 10/10 - Validation: 100%|██████████| 176/176 [03:41<00:00,  1.26s/it]


Epoch 10/10 - Train Loss: 0.2160, Train Accuracy: 0.9325, Valid Loss: 0.1751, Valid Accuracy: 0.9431


Testing: 100%|██████████| 118/118 [02:33<00:00,  1.30s/it]



Test Accuracy: 0.9435

Classification Report:
                           precision    recall  f1-score   support

    Asian-Green-Bee-Eater       0.97      0.92      0.95       150
      Brown-Headed-Barbet       0.94      0.85      0.89       150
             Cattle-Egret       0.98      0.93      0.95       150
        Common-Kingfisher       0.97      0.99      0.98       150
              Common-Myna       0.95      0.95      0.95       150
         Common-Rosefinch       0.91      0.89      0.90       150
        Common-Tailorbird       0.89      0.93      0.91       150
       Coppersmith-Barbet       0.92      0.93      0.92       150
           Forest-Wagtail       0.92      0.97      0.94       150
             Gray-Wagtail       0.95      0.98      0.97       150
                   Hoopoe       1.00      0.97      0.99       150
               House-Crow       0.95      0.92      0.93       150
     Indian-Grey-Hornbill       0.78      0.93      0.85       150
           Ind